In [ ]:
import torch.optim as optim
import glob
import json
import argparse
import os
import random
import math
from time import gmtime, strftime
from models import *
from dataset_GBU import FeatDataLayer, DATA_LOADER
from utils import *
from sklearn.metrics.pairwise import cosine_similarity
import torch.backends.cudnn as cudnn
import classifier

In [ ]:
dataset_name ='SUN'#(default) #help='dataset: CUB, AWA2, APY, FLO, SUN'
dataroot = 'C:/Sushree/Jio_Institute/Dataset/SDGZSL_data'#(default) #help='path to dataset'

workers = 1# type=int, help='number of data loading workers', default=1

image_embedding ='res101'#(default) # type=str
class_embedding ='att'#(default) #type=str

gen_nepoch = 400 # type=int, default=400, help='number of epochs to train for'
lr = 0.0001 # type=float, default=0.0001, help='learning rate to train generater'

zsl = False # type=bool, default=False, help='Evaluate ZSL or GZSL'
finetune = False # type=bool, default=False, help='Use fine-tuned feature'
ga = 15 # type=float, default=15, help='relationNet weight')
beta = 1 #', type=float, default=1, help='tc weight')
weight_decay = 1e-6 #', type=float, default=1e-6, help='weight_decay')
dis = 3#', type=float, default=3, help='Discriminator weight')
dis_step = 2#', type=float, default=2, help='Discriminator update interval')
kl_warmup = 0.01#', type=float, default=0.01, help='kl warm-up for VAE')
tc_warmup = 0.001#', type=float, default=0.001, help='tc warm-up')

vae_dec_drop = 0.5#', type=float, default=0.5, help='dropout rate in the VAE decoder')
vae_enc_drop = 0.4#', type=float, default=0.4, help='dropout rate in the VAE encoder')
ae_drop = 0.2#', type=float, default=0.2, help='dropout rate in the auto-encoder')

classifier_lr = 0.001#', type=float, default=0.001, help='learning rate to train softmax classifier')
classifier_steps = 50#', type=int, default=50, help='training steps of the classifier')

batchsize = 64#', type=int, default=64, help='input batch size')
nSample = 1200#', type=int, default=1200, help='number features to generate per class')

disp_interval = 200#', type=int, default=200)
save_interval = 10000#', type=int, default=10000)
evl_interval = 400#',  type=int, default=400)
evl_start = 0#',  type=int, default=0)
manualSeed = 5606#', type=int, default=5606, help='manual seed')

latent_dim = 20#', type=int, default=20, help='dimention of latent z')
q_z_nn_output_dim = 128#', type=int, default=128, help='dimention of hidden layer in encoder')
S_dim = 1024#', type=int, default=1024)
NS_dim = 1024#', type=int, default=1024)

#parser.add_argument('--gpu', default='0', type=str, help='index of GPU to use')
#opt = parser.parse_args()

if manualSeed is None:
    manualSeed = random.randint(1, 10000)
print("Random Seed: ", manualSeed)

np.random.seed(manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.cuda.manual_seed_all(manualSeed)

cudnn.benchmark = True
print('Running parameters:')
#print(json.dumps(vars(opt), indent=4, separators=(',', ': ')))
#opt.gpu = torch.device("cuda:"+opt.gpu if torch.cuda.is_available() else "cpu")


In [ ]:
train.py --dataset AWA2 --ga 0.5 --beta 1 --dis 0.3 --nSample 5000 --gpu 1 --S_dim 1024 --NS_dim 1024 --lr 0.00003 \
  --classifier_lr 0.003 --kl_warmup 0.01 --tc_warmup 0.001 --vae_dec_drop 0.5 --vae_enc_drop 0.4 --dis_step 2 \
  --ae_drop 0.2 --gen_nepoch 220 --evl_start 40000 --evl_interval 400 --manualSeed 6152